In [ ]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
from functools import lru_cache
from jdt import Jdt

In [ ]:
VIDEO_FILE = './heavy/2021-04-25.mp4'

In [ ]:
class GetFrame:
    # to improve: https://stackoverflow.com/questions/33650974/opencv-python-read-specific-frame-using-videocapture
    def __init__(self):
        self.cache = []
        self.videoCapture = None
        self.newVideoCapture()
    
    def newVideoCapture(self):
        self.release()
        self.videoCapture = cv.VideoCapture(VIDEO_FILE)
        self.cursor = 0
    
    def next(self):
        ret, frame = self.videoCapture.read()
        self.cursor += 1
        assert ret
        return frame
    
    def release(self):
        if self.videoCapture is not None:
            self.videoCapture.release()
    
    @lru_cache(maxsize=2)
    def __call__(self, frame_i, verbose=True):
        if frame_i < self.cursor:
            self.newVideoCapture()
        if verbose:
            j = Jdt(frame_i - self.cursor, UPP = 4)
        while self.cursor < frame_i:
            if verbose:
                j.acc()
            self.next()
        if verbose:
            j.complete()
        assert frame_i == self.cursor
        frame = self.next()
        swapped = frame.copy()
        swapped[:,:,0] = frame[:,:,2]
        swapped[:,:,2] = frame[:,:,0]
        return swapped

getFrame = GetFrame()

def widePlot(w = 16, h = 9):
    fig = plt.gcf()
    fig.set_size_inches(w, h)
def view(frame):
    plt.imshow(frame)
    widePlot()
    plt.show()

def channel(frame, z_keep):
    result = frame.copy()
    for z in range(3):
        if z != z_keep:
            result[:, :, z] = frame[:, :, z_keep] * .5
    return result
def normalize(frame):
    ceil  = np.max(frame)
    floor = np.min(frame)
    if 0 <= floor < 1 and 254 < ceil <= 255:
        return frame
    return np.rint((frame - floor) / (ceil - floor) * 255)
def whiten(frame):
    # convert one-channel frame to three-channel
    w, h = frame.shape
    result = np.zeros((w, h, 3), dtype=np.int16)
    frame = normalize(frame)
    result[:, :, 0] = frame
    result[:, :, 1] = frame
    result[:, :, 2] = frame
    return result

In [ ]:
# view(getFrame(100))

# A problem
focus/brightness variation!?

an outlier remover problem?  
in terms of overall brightness. 

focus? it is fine. everything is blurred anyways. 

In [ ]:
def eyeballOutlying():
    OUTLIER_PAGE = 8
    mid_i = round(OUTLIER_PAGE / 2)
    i = 1500
    getFrame(i)
    dev = []
    for _ in range(32):
        frames = [getFrame(i + x, False) for x in range(OUTLIER_PAGE)]
        brights = np.array([np.mean(f) for f in frames])
        median = sorted(brights)[mid_i]
        dev.extend(brights - median)
        print(i + np.where(brights < median - 8)[0])
        i += OUTLIER_PAGE
    plt.hist(dev)
eyeballOutlying()

In [ ]:
OUTLIER_PAGE = 8
BRIGHTNESS_THRES = 2

def OutlierRemoved():
    mid_i = round(OUTLIER_PAGE / 2)
    i = 0
    f = None
    while True:
        frames = [getFrame(i + x) for x in range(OUTLIER_PAGE)]
        i += OUTLIER_PAGE
        brights = [np.mean(f) for f in frames]
        median = sorted(brights)[mid_i]
        for j, b in enumerate(brights):
            if abs(b - median) < BRIGHTNESS_THRES:
                f = frames[j]
            if f is not None:
                yield f


In [ ]:
def LowPass(t_k = 5, xy_k = 5):
    oR = OutlierRemoved()
    history = [next(oR) for _ in range(t_k)]
    for frame in oR:
        a = np.sum(history) / t_k
        yield cv.medianBlur(a, xy_k)
        history.pop(0)
        history.append(frame)


In [ ]:
LowPass